# RAM-8,9 - SINGLE LABELED DATA ONLY
#### Jonathan Ramos 2/7/2024

There were some new images from RAM-9 added to the existing set I previously analzyed. Since the immunohistochemistry was done in the same batch, I can parse out these new data, add them to the existing *raw* set, and then normalize/count cells as usual. 

We have the following file naming scheme:
- _1.tif : cFos
- _2.tif : 8-oxo-DG --> EGFP
- _3.tif : mKate2

In [ ]:
import numpy as np
import pandas as pd
import glob